In [4]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [61]:
def leer_csv(path):
    df = pd.read_csv(path,  sep=';', header=5, index_col=0)
    df = df.drop(columns=['OBSERVACIONES', 'CONTABILIZADA','CÓDIGO_INTEGRIDAD', 'FECHA_HORA_PUBLICACIÓN', 'DIGITALIZACIÓN','DEPARTAMENTO', 'ID_DEPARTAMENTO', 'MUNICIPIO', 'ID_MUNICIPIO', 'CENTRO_DE_VOTACIÓN', 'PADRÓN'])
    return df

In [67]:
def llenar_nans(data):
    return data.applymap(lambda l: l if not pd.isna(l) else np.random.choice(100))
     

In [91]:
data_presidencial = llenar_nans(leer_csv('csv-pruebas\presidencial.csv'))

In [92]:
data_presidencial

,CUARENTA Y UNO,QUINCE,TREINTA Y DOS,TREINTA Y TRES,VEINTITRES,TREINTA Y SEIS,VEINTIOCHO,TREINTA,TREINTA Y CUATRO,DIECIOCHO,...,NULOS,BLANCO,EMITIDOS,INVÁLIDOS,TOTAL,IMPUGNACIONES,PRES-VOTAR,VALIDOS_CALCULADO,EMITIDOS_CALCULADO,TOTAL_CALCULADO
MESA,,,,,,,,,,,,,,,,,,,,,
1,44,25,33,26,87,37,46,94,95,28,...,75,39,71,18,80,57,61,19,15,94
2,75,11,64,46,38,94,47,17,22,59,...,31,17,53,38,37,0,59,24,48,46
3,36,14,73,11,9,53,48,95,20,62,...,65,31,81,20,48,4,95,5,93,66
4,64,83,49,76,7,7,40,71,9,56,...,30,3,84,92,32,29,46,90,7,63
5,67,85,81,32,73,65,16,11,66,24,...,66,66,38,42,46,72,89,93,79,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24581,69,86,9,44,72,4,15,29,1,19,...,63,18,17,37,90,69,84,19,63,94
24582,18,14,63,41,96,71,30,18,10,35,...,69,32,40,40,61,24,37,82,10,6
24583,41,2,81,94,76,73,93,43,8,32,...,82,1,63,93,80,26,79,38,30,83


In [ ]:
client = MongoClient("mongodb://localhost:27017/")
db = client["local"]
presidencial = db["mesas-presidencial"]

In [96]:
def convertir_a_mongo_docs(data):
    data_dict = data.transpose().to_dict()
    for mesa in data_dict:
       data_dict[mesa]['mesa'] = mesa
    return list(test_dict.values())

In [98]:
presidencial.insert_many(convertir_a_mongo_docs(data_presidencial))